In [31]:
 # import all the necessary libraries
import os
import re
import pandas as pd
import numpy as np

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta

import cartopy.crs as ccrs

from sklearn import datasets
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import discriminant_analysis
from sklearn import svm

# options
data_dir = os.getenv("HOME")+'/data/NOTAM-classifier'
pd.set_option('display.max_columns', 500)

# aestetics
plt.style.use('seaborn-whitegrid')
plt.rc('pdf',fonttype=42)
sns.mpl.rc('figure', figsize = (10, 8))
sns.set_context('notebook', font_scale=1.8, rc={'lines.linewidth': 2.5})

# File contents

In [35]:
NOTAMs_df = pd.read_csv(data_dir+'/NOTAMS.csv', sep=',').set_index('id')

select = NOTAMs_df['rvid'] > 1
NOTAMs_df = NOTAMs_df[select]

In [36]:
print(len(NOTAMs_df))

75159


## Header

In [37]:
NOTAMs_df.head(10)

,good,txt,supress,rvid,scope,FIR_12,high_min_alt,low_max_alt,diurnal_duration,long_text,small_radius,trafficind,code_23,n_locations,code_45
id,,,,,,,,,,,,,,,
A3621/18,True,"ACT MIL AREAS MMR-116, MMR-117 AND MMR-119",0,1698,W,MM,False,True,False,False,False,IV,RR,1.0,CA
A3657/18,True,AIRBORNE WARNING AND CONTROL SYSTEM FLIGHT WIL...,1,1698,W,LT,True,False,False,True,False,IV,WE,2.0,LW
A3620/18,True,RESTRICTED AREA LATERAL LIMIT: CIRCLE OF 2...,1,1503,W,MM,False,True,False,True,True,IV,RR,1.0,CA
A3566/18,True,AERODROME CLSD EXCEPT HELIPADS,0,1598,A,ED,False,False,False,False,False,IV,FA,1.0,LC
A3478/18,True,RWY 16R/34L CLS,0,1598,A,KZ,False,False,False,False,False,IV,MR,1.0,LC
B1780/18,True,AREA SPECIFIED AS BELOW PROHIBITED FOR IFR/VFR...,1,1698,W,LT,False,True,False,True,False,IV,RT,1.0,LP
E4203/18,True,TWY E(BTN E8 AND J) E9-CLSD DUE TO CONS,0,1698,A,RJ,False,False,True,False,False,IV,MX,1.0,LC
F1751/18,True,UA (25KG) OPR PSN S33 40.2 E150 51.2 (RIVERSTO...,1,1698,AW,YM,False,True,True,True,True,IV,WU,1.0,LW
C1957/18,True,IAC1 ILS Z RWY05 NO AVBL DEBIDO A GP U/,1,1503,A,SC,False,False,False,False,False,I,PI,1.0,AU


In [38]:
features = [
    'scope', 'FIR_12', 'high_min_alt', 
    'low_max_alt', 'diurnal_duration', 
    'long_text', 'small_radius', 
    'trafficind', 'code_23', 
    'n_locations', 'code_45']

In [39]:
from sklearn import preprocessing

n_samples = len(NOTAMs_df)
n_features = len(features)
X = np.zeros((n_samples, n_features))

le = preprocessing.LabelEncoder()
for i,feature in enumerate(features):
    print('Encoding {}'.format(feature))    
    X[:, i] = le.fit_transform(NOTAMs_df[feature].astype(str))

y = NOTAMs_df['supress']

X

Encoding scope
Encoding FIR_12
Encoding high_min_alt
Encoding low_max_alt
Encoding diurnal_duration
Encoding long_text
Encoding small_radius
Encoding trafficind
Encoding code_23
Encoding n_locations
Encoding code_45


array([[  5.,  81.,   0., ..., 131.,   0.,  10.],
       [  5.,  73.,   1., ..., 147.,   1.,  38.],
       [  5.,  81.,   0., ..., 131.,   0.,  10.],
       ...,
       [  0.,  71.,   0., ...,  94.,   0.,  27.],
       [  0., 163.,   0., ...,  89.,   0.,  40.],
       [  0.,  54.,   0., ...,  94.,   0.,  23.]])

In [40]:
def classify(X, y, classifier, prob=None, random_seed=20091982):
    """ Run classifier and print
    results
    """
    
    X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(
        X, y, test_size=0.20, random_state=random_seed)
    
    classifier.fit(X_train, y_train)
    if prob is not None:
        y_pred = classifier.predict_proba(X_test)[:,0] < prob
    else:
        y_pred = classifier.predict(X_test)
        
 
    N = len(y_test)
    TP = np.sum((y_pred == y_test) & (y_test == 1))
    TN = np.sum((y_pred == y_test) & (y_test == 0))
    FP = np.sum((y_pred != y_test) & (y_pred == 1))
    FN = np.sum((y_pred != y_test) & (y_pred == 0))

    accuracy = (TP+TN)/N
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

    result_string = 'N={0}, TP={1}, TN={2}, FP={3}, FN={4}\n'.format(N, TP, TN, FP, FN)
    result_string += \
        'Precision: {0:.4f}, recall: {1:.4f}, accuracy: {2:.4f}'\
        .format(precision, recall, accuracy)

    print(result_string)
    
    report = metrics.classification_report(y_test, y_pred)
    
    
    print(report)
    
    return classifier

def get_feature_importances(cols, importances):
    
    count = 0
    indices = np.argsort(importances)[::-1]
    for i in indices:
        print('{1}: {0:.2f}%'.format(
            importances[i]*100.0, cols[i]))
        count += 1
        #if count == 10:
        #    break
    return

In [41]:
classifier = classify(X, y, RandomForestClassifier(random_state=20091982), prob=0.5)

# recall = 98%, reduce screened NOTAMs by 90%
classifier = classify(X, y, RandomForestClassifier(random_state=20091982), prob=0.1)

# recall = 99%, reduce screened NOTAMs by 80%
classifier = classify(X, y, RandomForestClassifier(random_state=20091982), prob=0.001)

N=15032, TP=5922, TN=7697, FP=671, FN=742
Precision: 0.8982, recall: 0.8887, accuracy: 0.9060
             precision    recall  f1-score   support

          0       0.91      0.92      0.92      8368
          1       0.90      0.89      0.89      6664

avg / total       0.91      0.91      0.91     15032

N=15032, TP=4714, TN=8282, FP=86, FN=1950
Precision: 0.9821, recall: 0.7074, accuracy: 0.8646
             precision    recall  f1-score   support

          0       0.81      0.99      0.89      8368
          1       0.98      0.71      0.82      6664

avg / total       0.89      0.86      0.86     15032

N=15032, TP=4063, TN=8320, FP=48, FN=2601
Precision: 0.9883, recall: 0.6097, accuracy: 0.8238
             precision    recall  f1-score   support

          0       0.76      0.99      0.86      8368
          1       0.99      0.61      0.75      6664

avg / total       0.86      0.82      0.81     15032



## Features importances

In [52]:
get_feature_importances(features, classifier.feature_importances_)

AttributeError: 'MLPClassifier' object has no attribute 'feature_importances_'

## One hot encoders

In [43]:
enc = preprocessing.OneHotEncoder()    
X_one_hot = enc.fit_transform(X)

X_one_hot.shape

(75159, 415)

In [44]:
# essentially recall = 100%, reduce screened NOTAMs by 60%
classifier = classify(X_one_hot, y, LogisticRegression(), prob=0.1)

N=15032, TP=4107, TN=8248, FP=120, FN=2557
Precision: 0.9716, recall: 0.6163, accuracy: 0.8219
             precision    recall  f1-score   support

          0       0.76      0.99      0.86      8368
          1       0.97      0.62      0.75      6664

avg / total       0.86      0.82      0.81     15032



In [45]:
# some more test
classifier = classify(X_one_hot, y, LogisticRegression(), prob=0.01)

N=15032, TP=1591, TN=8352, FP=16, FN=5073
Precision: 0.9900, recall: 0.2387, accuracy: 0.6615
             precision    recall  f1-score   support

          0       0.62      1.00      0.77      8368
          1       0.99      0.24      0.38      6664

avg / total       0.79      0.66      0.60     15032



In [46]:
classifier = classify(X_one_hot, y, MLPClassifier(), prob=0.5)

N=15032, TP=6091, TN=7566, FP=802, FN=573
Precision: 0.8837, recall: 0.9140, accuracy: 0.9085
             precision    recall  f1-score   support

          0       0.93      0.90      0.92      8368
          1       0.88      0.91      0.90      6664

avg / total       0.91      0.91      0.91     15032



In [47]:
classifier = classify(X, y, RandomForestClassifier(random_state=20091982), prob=0.5)
classifier = classify(X_one_hot, y, RandomForestClassifier(random_state=20091982), prob=0.5)

N=15032, TP=5922, TN=7697, FP=671, FN=742
Precision: 0.8982, recall: 0.8887, accuracy: 0.9060
             precision    recall  f1-score   support

          0       0.91      0.92      0.92      8368
          1       0.90      0.89      0.89      6664

avg / total       0.91      0.91      0.91     15032

N=15032, TP=5939, TN=7725, FP=643, FN=725
Precision: 0.9023, recall: 0.8912, accuracy: 0.9090
             precision    recall  f1-score   support

          0       0.91      0.92      0.92      8368
          1       0.90      0.89      0.90      6664

avg / total       0.91      0.91      0.91     15032



In [48]:
classifier = classify(X_one_hot, y, BernoulliNB(), prob=0.01)

N=15032, TP=4127, TN=7756, FP=612, FN=2537
Precision: 0.8709, recall: 0.6193, accuracy: 0.7905
             precision    recall  f1-score   support

          0       0.75      0.93      0.83      8368
          1       0.87      0.62      0.72      6664

avg / total       0.81      0.79      0.78     15032



In [49]:
# classifier = classify(X, y, GradientBoostingClassifier(), prob=0.01)
classifier = classify(X, y, svm.SVC(probability=False), prob=None)

N=15032, TP=5748, TN=7738, FP=630, FN=916
Precision: 0.9012, recall: 0.8625, accuracy: 0.8972
             precision    recall  f1-score   support

          0       0.89      0.92      0.91      8368
          1       0.90      0.86      0.88      6664

avg / total       0.90      0.90      0.90     15032



In [50]:
# Takes time to train
# classifier = classify(X, y, svm.SVC())


In [51]:
classifier = classify(
    X_one_hot, y, 
    MLPClassifier(
        hidden_layer_sizes=(200, 10), 
        learning_rate_init=0.01, 
        verbose=True), 
    prob=0.5)

Iteration 1, loss = 0.25088667
Iteration 2, loss = 0.20333454
Iteration 3, loss = 0.19151623
Iteration 4, loss = 0.18229264
Iteration 5, loss = 0.17718971
Iteration 6, loss = 0.17389192
Iteration 7, loss = 0.16981423
Iteration 8, loss = 0.16785128
Iteration 9, loss = 0.16587330
Iteration 10, loss = 0.16414383
Iteration 11, loss = 0.16233877
Iteration 12, loss = 0.16088436
Iteration 13, loss = 0.15942257
Iteration 14, loss = 0.15894861
Iteration 15, loss = 0.15779095
Iteration 16, loss = 0.15674014
Iteration 17, loss = 0.15748028
Iteration 18, loss = 0.15509108
Iteration 19, loss = 0.15526681
Iteration 20, loss = 0.15392364
Iteration 21, loss = 0.15446527
Iteration 22, loss = 0.15360426
Iteration 23, loss = 0.15298549
Iteration 24, loss = 0.15218380
Iteration 25, loss = 0.15174282
Iteration 26, loss = 0.15199035
Iteration 27, loss = 0.15172909
Iteration 28, loss = 0.15066149
Iteration 29, loss = 0.15151673
Iteration 30, loss = 0.15149025
Iteration 31, loss = 0.14990012
Iteration 32, los